In [54]:
# !pip install transformers accelerate
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
# !pip install datasets
from datasets import load_metric
# !pip install evaluate
from evaluate import evaluator
from sklearn.model_selection import train_test_split
import csv

# notebook_login()

CUDA_LAUNCH_BLOCKING=1


In [55]:
# from huggingface_hub import notebook_login
# notebook_login()

In [56]:
# df = pd.read_csv('D:\IET software-CC-CL\Commit Classification\experiment\Commit_dataset.csv', encoding="cp1252")

# label2id = {'a':'Adaptive','p':'Perfective','c':'Corrective'}
label2id = {'Corrective':0,'Adaptive':1,'Perfective':2}
# df = df.replace({"3_labels": label2id})
# df = pd.read_csv(r'dataset.csv',engine="python")
# df['text'] = "['DIFF]" + df['diff']
# # df = df.replace({"2_labels": label2id})
# print(df)

train = pd.read_csv('train.csv',index_col=0)
# train = train.rename(columns={'3_labels':'label','comment':'text'})
train = train.replace({"label": label2id})
train.fillna(0, inplace=True)
test = pd.read_csv('test.csv',index_col=0)
# test = test.rename(columns={'3_labels':'label','comment':'text'})
test = test.replace({"label": label2id})
test.fillna(0, inplace=True)
print(len(test))



1722


In [57]:
# from datasets import Dataset, load_metric

In [58]:
# train_dataset = Dataset.from_pandas(train)
# test_dataset = Dataset.from_pandas(test)

In [59]:
len(train)

71

In [60]:
train['label'].value_counts()

0    24
2    24
1    23
Name: label, dtype: int64

In [61]:
test['label'].value_counts()

0    579
2    576
1    567
Name: label, dtype: int64

In [62]:
# !pip install sentence_transformers
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-mpnet-base-v2')

In [63]:
train['embeddings'] = train['text'].apply(model.encode)
test['embeddings'] = test['text'].apply(model.encode)

In [64]:
test

,commit_id,project,text,label,ADDING_ATTRIBUTE_MODIFIABILITY,ADDING_METHOD_OVERRIDABILITY,ADDITIONAL_CLASS,ADDITIONAL_FUNCTIONALITY,ADDITIONAL_OBJECT_STATE,ALTERNATIVE_PART_DELETE,...,Inline Variable,Parametrize Variable,Rename Variable,Replace Variable with Attribute,Change Variable Type,Move and Rename Class,Move Class,Extract and Move Method,Move Source Folder,embeddings
1782,337192523cb453b134d6279f18d9edf4ef5f40aa,apache$hama,Optimize memory use.\ngit-svn-id: https://svn....,2,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,"[0.05318065, 0.14155823, -0.075034715, 0.12697..."
1402,8048657e265ab4f39b8b2e42425602bf151c7b91,Delta Spike,DELTASPIKE-745 fix cdictrl-weld to be able to ...,0,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,"[-0.18996559, -0.005256058, 0.025005754, -0.03..."
890,e3d1a1dda22723fc896bfc96c6db57c500faf208,spring-framework,@Resource injection points support @Lazy as we...,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,"[-0.060315046, -0.26188236, -0.036801048, 0.05..."
777,d52a7d799c2ab73dbc38de272d173bbe9cf08797,orientdb,Fixed issue reported by Steven about TX- congr...,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,"[-0.010595558, 0.009616613, -0.026825245, 0.04..."
1785,52ebe9ed4e45755f54da430b92d194774b161aa8,crosswire$jsword,JS-264 Modify Indexing for low memory devices\n,2,0,0,1,1,1,1,...,0,0,1,0,0,0,0,0,0,"[0.061383214, 0.020680219, -0.18614127, 0.0402..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1119,77f3ff5cfc41f43bc51c3ce4505ff1a1aa80e2f1,Search_api,Issue #1064520 by drunken monkey: Added a proc...,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[-0.095308304, 0.23708284, 0.022497611, 0.0464..."
980,30c774c06fa73105ee30032a4f967dce95b55f1f,Valadoc,gir-importer: add support for markdown\n,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[-0.0847115, 0.2893154, -0.043920513, -0.02397..."
594,55ff4ae941e12c170a5e49801e599827e6461e27,kotlin,Abstract Lazy test for gianostics--,2,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,"[-0.17942427, 0.31981316, -0.13050508, 0.17941..."
1154,f932807d5c49548c1bbbdc6df1e949fe196e6cc5,tapiji,adds the plug-ins and\n,1,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,"[-0.10518667, -0.11359331, -0.110450536, 0.015..."


In [65]:
X_train = train['embeddings'].to_list()
X_test = test['embeddings'].to_list()

y_train = train['label'].to_list()
y_test = test['label'].to_list()

In [66]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(X_train,y_train)

LogisticRegression()

In [67]:
from sklearn import metrics
predicted = LR.predict(X_test)
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted,average='weighted'))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted,average='weighted'))
print("Logistic Regression Recall:",metrics.f1_score(y_test, predicted,average='weighted'))

Logistic Regression Accuracy: 0.5720092915214866
Logistic Regression Precision: 0.5720538967277682
Logistic Regression Recall: 0.5720092915214866
Logistic Regression Recall: 0.5717638580806262
